In [1]:
import pandas as pd
import openpyxl as xl
from shutil import copy
from openpyxl.styles import Alignment
from isdayoff import ProdCalendar, DayType
from datetime import date, timedelta

calendar = ProdCalendar(locale='ru') #добавляем производственный календарь с Российскими выходными/праздниками


staff = pd.read_excel('Штат08.10.2021.xls')  # считываем файл со штатом сотрудников


staff['full_name_dep']=(staff["Фамилия"] + " " + staff["Имя"] + " " + staff["Отчество"] + " " + staff["Штатная должность"]).astype("string") 
#делаем full_name - ФИО + департамент где работает

DEPARTMENTS = staff['Подразделение (наименование)'].unique() 
#находим все уникальные департменты в столбце "Подразделение(наименование)"

TEMPLATE_SIZE = 100 # ppl per file(кол-во людей, которое может быть в одном файле, если окажется что в департменте работает больше, просто изменить значение)
FIRST_ROW = 18     #первая строчка, откуда начинается заполнение табеля сотрудниками 

#здесь как раз и отлавливаю ошибку, если вдруг окажется что лист очень маленький, а народу в департменте много
MAX_PPL = staff.groupby('Подразделение (наименование)')['full_name_dep'].count().max()
#ppl = people
if TEMPLATE_SIZE < MAX_PPL:
    raise ValueError('Шаблон табеля меньше, чем кол-во людей в каком-то департаменте!\n Для этого нужно взять шаблон(TableForm.xlsx) и добавить туда примерно +50% от текущего (100=>150=>225 etc)')

LAST_ROW = FIRST_ROW + TEMPLATE_SIZE*4 - 1  # последняя пустая строчка в  шаблоне табеля

In [2]:
"""
cell height for the name of the person signing the document = CHFTNOTPSTD = CTD
высота ячейки, для названия должности сотрудника, который будет подписывать документ
данную величину можно регулировать по усмотрению(x пикселей)
------------------------------------------------
jump - через если клетка слита через 4 ячейки, т это значит что после функии нужно идти на 4 вниз, к следующей
start_col - первый столбец, откуда все и начинается 
last_col - последний столбец
"""
def unmerge_cells_fun(jump, start_col, last_col):
        row = r
        for row in range(row, LAST_ROW, jump):
            for col in[(start_col,last_col)]:
                sheet.unmerge_cells(start_row=row, start_column=col[0], end_row=row + jump - 1, end_column=col[1])
                 
def cell_function_days_U(cell_num_days): #добавление отработанных дней в колонку U
        days_cell = str("U") + str(cell_num_days)       #создаем номер ячейки, куда будем заполнять данные
        return days_cell

    
def cell_function_hours_U(cell_num_hours):  #добавление отработанных часов в колонку U
        hours_cell = str("U") + str(cell_num_hours)
        return hours_cell
    
    
def cell_function_days_V(cell_num_days): #добавление отработанных дней в колонку V
        days_cell_2 = str("V") + str(cell_num_days)  #создаем номер ячейки, куда будем заполнять данные
        return days_cell_2
    
    
def cell_function_hours_V(cell_num_hours): #добавление отработанных часов в колонку V
        hours_cell_2 = str("V") + str(cell_num_hours) 
        return hours_cell_2
    

"""    
start_col - первый столбец, откуда все и начинается 
last_col - последний столбец
fpos_row -первая строка, где начинаем соединять 
lpos_row - послдная строка, где заканчиваем соединять 
k - некоторые вещи нам нужно соединить один раз(без цикла), и удобнее просто указать их место
"""        
def merge_cells_fun(start_col, last_col, fpos_row, lpos_row, k):
    row = r + k
    for col in[(start_col,last_col)]:
        sheet.merge_cells(start_row=row + fpos_row, start_column=col[0], end_row=row + lpos_row, end_column=col[1])

'''
все что внизу - наработчки, которые пока не используются
соединяю поля, куда будут вбиваться подписи, и заоодно сраззу эти подписи
пока что не работает
'''
def start_col_fun(start_col):
    start_col_fun_2 = str(chr(64+start_col))
    return start_col_fun_2

def last_col_fun(last_col):
    last_col_fun_2 = str(chr(64+last_col))
    return last_col_fun_2

        
def merge_cells_fun_inside(start_col, last_col, fpos_row, lpos_row, k,text):
    row = r + k
    start_col_first_pos = start_col
    
    for col in[(start_col,last_col)]:
        sheet.merge_cells(start_row=row + fpos_row, start_column=col[0], end_row=row + lpos_row, end_column=col[1]) 
    
    top_left_cell = sheet[f'{start_col_fun(start_col_first_pos)}{r+k}'] 
    top_left_cell.value = text 
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")      
    

In [3]:
for DEPARTMENT in DEPARTMENTS:
    
    #создаем файл для каждого департмента 
    file = f'{DEPARTMENT}.xlsx'
    #copy('TableFormCount.xlsx', file) #название можно поменять
    copy('TableForm.xlsx', file) #название можно поменять
    wb = xl.load_workbook(file,data_only=True)
    sheet = wb['Табель'] #название листа, можно поменять на нужный в шаблоне на входе (то есть в TableForm.xlsx)
        
    top_left_cell = sheet['B5'] #обозначаем клетку, куда хотим добавить название департмента
    top_left_cell.value = DEPARTMENT #добавляем название
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")    #выравниваем по цетру
    
    # добавляем всех сотрудников в файл
    people = staff.loc[staff["Подразделение (наименование)"] == DEPARTMENT,                # фильтр по отделу
                                            ['full_name_dep', 'Табельный номер']] # столбцы которые нам нужны
    people = people.sort_values('full_name_dep')
    n = people.shape[0] # n = кол-во людей 
    r = FIRST_ROW       # r = текущая строчка    
    
    cell_num_days = 18           #номер ячейки, сколько дней сотрудник появлялся на работе или работал дистанционно 
    cell_num_hours = 19        #номер ячейки, куда мы будем добавлять сумму всех отработанных часов
    numbering = 1                   #нумерация людей в списке 
    count_of_work_days = 0   #счетчкий, отвечающий за подсчет рабочих дней у сотруднкика 
    count_of_work_hours = 0 #счетчкий, отвечающий за подсчет рабочих часов у сотруднкика 
           
    for person in people.itertuples():
        sheet.cell(r, 3).value = person[1]   # ФИО + должность 
        sheet.cell(r, 4).value = person[2]   # Табельный номер
        sheet.cell(r, 2).value = numbering   #нумерация сотрудников в файле 
        numbering += 1
        
        day_in_the_month = 1 #счетчик дня в месяце
        amount_days_in_the_month = 30
        #кол-во дней в месяце. Пока не придумал как автомотизировать, чтобы прога сама выбирала кол-во дней в зависимости от месяца
        
        column_for_work_days = 5 #столбец, откуда мы начинаем проставлять рабочие/выходные дни 
        
        days_count=1 #идет счет дней, чтобы после 15-ого дня перейти на новую строчку для второй половины месяца
        if (file != "Отдел информационных технологий.xlsx"):
            for i in range(1, amount_days_in_the_month): #весь месяц, от первого до последнего дня
            
                while(days_count < 15+1): #первая половина
                        if calendar.check(date(2021, 11, day_in_the_month))== DayType.WORKING: #проверяем, является ли день x рабочим
                            sheet.cell(r,column_for_work_days).value = "Я"
                            sheet.cell(r+1,column_for_work_days).value = 8
                        else:                      #если не рабочий, то =>
                            sheet.cell(r,column_for_work_days).value = "В"
                            sheet.cell(r+1,column_for_work_days).value = 0
                        day_in_the_month+=1 
                        column_for_work_days+=1
                        days_count+=1
                        
                column_for_work_days = 5 #начинаем с 5-ого столбца, идем до 22
            
                while(days_count<30+1): #тоже самое, только для второй половины
                    if calendar.check(date(2021, 11, day_in_the_month))== DayType.WORKING:
                        sheet.cell(r+2,column_for_work_days).value = "Я"
                        sheet.cell(r+3,column_for_work_days).value = 8
                    else:
                        sheet.cell(r+2,column_for_work_days).value = "В"
                        sheet.cell(r+3,column_for_work_days).value = 0              
                    day_in_the_month+=1
                    days_count+=1
                    column_for_work_days+=1 
        
        else:
            for i in range(1, amount_days_in_the_month): #весь месяц, от первого до последнего дня
            
                while(days_count < 15+1): #первая половина
                        if calendar.check(date(2021, 11, day_in_the_month))== DayType.WORKING: #проверяем, является ли день x рабочим
                            sheet.cell(r,column_for_work_days).value = "Я"
                            sheet.cell(r+1,column_for_work_days).value = 12
                        else:                      #если не рабочий, то =>
                            sheet.cell(r,column_for_work_days).value = "В"
                            sheet.cell(r+1,column_for_work_days).value = 0
                        day_in_the_month+=1 
                        column_for_work_days+=1
                        days_count+=1
                        
                column_for_work_days = 5 #начинаем с 5-ого столбца, идем до 22
            
                while(days_count<30+1): #тоже самое, только для второй половины
                    if calendar.check(date(2021, 11, day_in_the_month))== DayType.WORKING:
                        sheet.cell(r+2,column_for_work_days).value = "Я"
                        sheet.cell(r+3,column_for_work_days).value = 12
                    else:
                        sheet.cell(r+2,column_for_work_days).value = "В"
                        sheet.cell(r+3,column_for_work_days).value = 0              
                    day_in_the_month+=1
                    days_count+=1
                    column_for_work_days+=1 
      
                                
        #обнуляем счетчики, чтобы потом применить их на след человеке                   
        day_in_the_month = 1 
        days_count = 1
        column_for_work_days = 1   
          
       #берем 'r' как первую строчку  
        for i in sheet[f'E{r}:T{r}'][0]:   #берем строчку, где написано, был/не был на работе,  и т.п.
            Workday = i.value                                                           #берем  первый элемент и проверяем 
            if Workday == 'ДР':
                count_of_work_days += 1
            if Workday == 'Я':
                count_of_work_days += 1    
        
        # добавляем в отведенные ячейки сотрудника кол-во отработанных дней
        sheet[cell_function_days_U(cell_num_days)] = count_of_work_days  
        
        for i in sheet[f'E{r+1}:T{r+1}'][0]:   
            if(i.value !=None): #в месяце могут быть не все 31, а значит могут быть пустые клетки, тобиш None, их складывать нельзя
                Workday = i.value                                                           
                count_of_work_hours += Workday
                
                
        sheet[cell_function_hours_U(cell_num_hours)] = count_of_work_hours
        
        count_of_work_hours_half = count_of_work_hours #считаем кол-во рабочих часов для первых 15 дней и "запоминаем их"
        count_of_work_hours = 0 #обнуляем счетчик, чтобы посчитать вторую половину
        

        count_of_work_days_for_first_half  = count_of_work_days #считаем кол-во рабочих дней для первых 15 дней и "запоминаем их"
        count_of_work_days = 0 #обнуляем счетчик, чтобы посчитать вторую половину
        
        
        for i in sheet[f'E{r+2}:T{r+2}'][0]:   
            Workday = i.value                                                           
            if Workday == 'ДР':
                count_of_work_days += 1
            if Workday == 'Я':
                count_of_work_days += 1   
        
        #заполняем все нужные клетки
        sheet[cell_function_days_U(cell_num_days+2)] = count_of_work_days 
        
        
        
        for i in sheet[f'E{r+3}:T{r+3}'][0]:   
            if(i.value !=None):
                Workday = i.value                                                           
                count_of_work_hours += Workday
                
        sheet[cell_function_hours_U(cell_num_hours+2)] =  count_of_work_hours 
        sheet[cell_function_hours_V(cell_num_hours + 1)] = count_of_work_hours + count_of_work_hours_half
        count_of_work_hours = 0
            
 
        sheet[cell_function_days_V(cell_num_days)] = count_of_work_days + count_of_work_days_for_first_half
        count_of_work_days  = 0
    
        #переход на следующею ячейку, то есть на следуещего сотрудника по списку
        cell_num_days += 4 
        cell_num_hours += 4
        count_of_work_days  = 0 # обнуляем счетчик рабочих дней сотрудника, чтобы посчитать их для следуещего 
        r += 4 #добавляем через 4 клетки нового сотрудника 
    
    
    
    row = r #возвращеем row в изначальное состояние, чтобы сделать тотже цикл с другими объектами
    
    unmerge_cells_fun(2,22,22)            
    unmerge_cells_fun(4,2,2)
    unmerge_cells_fun(4,3,3)
    unmerge_cells_fun(4,4,4)
    
    
    
       
   
    merge_cells_fun(22,22,0,0,0)
    
    merge_cells_fun(2,3,1,1,0)
    merge_cells_fun(4,5,1,1,0)
    #merge_cells_fun_inside(4,5,1,1,1,text)
    merge_cells_fun(7,9,1,1,0)
    merge_cells_fun(11,19,1,1,0)
    merge_cells_fun(23,24,1,1,0)
    merge_cells_fun(28,29,1,1,0)
    merge_cells_fun(23,24,4,4,0)
    merge_cells_fun(28,29,4,4,0)
    
    
    
    merge_cells_fun(4,5,0,0,2)
    merge_cells_fun(7,9,0,0,2)
    merge_cells_fun(11,19,0,0,2)
    merge_cells_fun(23,24,0,0,5)
    merge_cells_fun(23,24,0,0,2)
    merge_cells_fun(28,29,0,0,2) 
    merge_cells_fun(28,29,0,0,5)
    merge_cells_fun(21,22,1,1,0)
    merge_cells_fun(21,22,1,1,3)
   
    
    
    CTD = 33 #в функциях в самом начале написано что это
    
    rd = sheet.row_dimensions[r+1]   #расширяем ячейку для "Ответственное лицо" и "Руководитель структурного подразделения"
    rd.height = CTD              
    rd1 = sheet.row_dimensions[r+4]  #расширяем ячейку для "Работник кадровой службы"
    rd1.height = CTD            
     
    sheet.delete_rows(r, LAST_ROW + 1 - r) # удаляем неиспользованные строки
    
    #корректируем print area чтобы она соотвестовала финальному результату, который нам нужен (без пустых пропусков и т.п)
    sheet.print_area = sheet.calculate_dimension()
    
    wb.save(file)   #сохраняем все изменения 
    
    # нужен для теста (сделает один файл), удалите break если надо чтобы сделало все департаменты (весь цикл)
    break
   